In [1]:
import numpy as np
import tensorflow as tf

# Overview
* https://github.com/ageron/handson-ml2

https://www.tensorflow.org/guide/effective_tf2
Use Python to define computational graph, which is exectued in C++ Code.
Due to the graphical structure TF is highly parallizeable.
TF can compute gradients automatically.
The main python API is very flexible at the cost of higher complexity.
It comes with tensorboard: a visualization tool for the computational graph.

### Tensor flow APIs for machine learning
* `tensorflow.estimator`: API for predefined models (very simalar to scikit-learn)
* `tensorflow.losses`: Losses
* `tensorflow.metrics`: Metrics
* `tensorflow.layes`: Neural network layes



### High level APIs to Tensorflow
* todo

### Rought Structure of Tensorflow
In principle (as e.g., in Spark) a tensorflow computation consists of two steps:
1. **Construction phase:**
 This parts specifies the computational graph. I.e., it specifies the model to compute
 In tensorflow we call the associated object _dataflow graph_ (or just graph). Any additional variable is just a node added to the graph. The individual nodes are **Tensors** 
 and **operations**. Tensors don't carry any data. 
 
2. **Execution phase:**
* This part then runs the actual computation and returns results. 
* In Tensorflow the associated object is called a _session_. The return dypes of a session are **numpy arrays**.


**In TF 2 this has changed as eager exectution is put forward (so there is no more the need to work with the `session` context as in TF 1)**




# [Datatypes](https://www.tensorflow.org/tutorials/customization/basics)
* Basic building block are `tf.Tensor`
* Very similar to `np.ndarray`
* However there are also differences
    - are **immutable**
    - _can_ live in accelerator memory: GPU and TPU (Tensor processing unit)
    
    
### Operations on Tensors
- a rich library over tensors https://www.tensorflow.org/guide/effective_tf2. In particular:
- Tensor manipulation: masking, broadcasting, ...
- https://www.tensorflow.org/api_docs/python/tf/function
- [Linear algebra](https://www.tensorflow.org/api_docs/python/tf/linalg)
- [Mathematical functions](https://www.tensorflow.org/api_docs/python/tf/math)
- [Fourier tranformations](https://www.tensorflow.org/api_docs/python/tf/signal)
- [Statistical operations](https://www.tensorflow.org/api_docs/python/tf/random)
- [Sparse tensor representation](https://www.tensorflow.org/api_docs/python/tf/sparse)
- [String operations](https://www.tensorflow.org/api_docs/python/tf/strings)
- [But you can also make your costum ops in C++](https://www.tensorflow.org/guide/create_op)
   



### Compatibility with numpy

* `tf.Tensor` $\rightarrow$ `np.ndarray` 
    - Numpy operations automatically convert Tensors to  NumPy ndarrays
    - explicitly with the `numpy()` method on a Tensor
* `tf.Tensor` $\leftarrow$ `np.ndarray` 
    - TensorFlow operations automatically convert NumPy ndarrays to Tensors

In [2]:
numpy_array = np.array([[1,2,3], [1,2,3]])
tensor = tf.constant(numpy_array)

# implicit conversion of ndarray -> tensor within tf function
tf.print(tf.math.reduce_sum(numpy_array, axis=1))

# implicit conversion tensor -> ndarray within numpy function
print(np.sum(tensor, axis=1)) 

# explicit conversion tensor -> nd.arrray
print(tensor.numpy()) 

[6 6]
[6 6]
[[1 2 3]
 [1 2 3]]


### Tensors
> A Tensor is a symbolic handle to one of the outputs of an Operation. 
It does not hold the values of that operation's output, but instead provides a means of computing those values.
A `tf.Tensor` object represents a partially defined computation that will eventually produce a value. TensorFlow programs work by first building a graph of tf.Tensor objects, detailing how each tensor is computed based on the other available tensors and then by running parts of this graph to achieve the desired results.

A Tensor has always a 
* datatype
* shape

Tensors can be:
* reshaped
* sliced 
* cast to other datatypes
* broadcasted
* [see notebook](https://colab.research.google.com/notebooks/mlcc/creating_and_manipulating_tensors.ipynb?utm_source=mlcc&utm_campaign=colab-external&utm_medium=referral&utm_content=tensors-colab&hl=de)

There Are several types of tensors

* `tf.Variable`
* `tf.constant`
* `tf.placeholder`
* `tf.SparseTensor`

* Apart from `tf.Variable`, Tensors are _immuatable_



> If there's a tf.device scope active, the variable will be placed on that device; otherwise the variable will be placed on the "fastest" device compatible with its dtype (this means most variables are automatically placed on a GPU if one is available). For example, the following snippet creates a variable named v and places it on the second GPU device:


~~~~(.python)
with tf.device("/device:GPU:1"):
  v = tf.Variable(tf.zeros([10, 10]))
~~~~

In [57]:
vec = tf.constant([1,2,3], tf.float32)
var = tf.Variable(vec)

# variables are mutable: 
var.assign_add(tf.ones(3))

# get the rank (=number of dimensions)
tf.rank(vec)

x = tf.ones([2,2,2], tf.int16)

# reshape
tf.reshape(x, [4,-1])

# cast
tf.cast(x, tf.float32)

# slicing
x[1,:,:] 

# broadcasting
x * 10


<tf.Tensor: shape=(2, 2, 2), dtype=int16, numpy=
array([[[10, 10],
        [10, 10]],

       [[10, 10],
        [10, 10]]], dtype=int16)>

### Ragged Tensors

# Data input pipelines with [Data Sets](https://www.tensorflow.org/guide/data)
* Build data pipelines to feed model
* It consists of 3 Steps:
    - Create _source_ dataset (i.e. reading slices from a tensor of from a text filed)
    - Apply transformation (`map`, `shuffle`, `batch`) 
    - Iterate

In [3]:
# create source dataset 
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])
# apply transformations
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)
# iterate over input data for model
for t in ds_tensors:
    print(t)

tf.Tensor([4 9], shape=(2,), dtype=int32)
tf.Tensor([ 1 25], shape=(2,), dtype=int32)
tf.Tensor([36 16], shape=(2,), dtype=int32)


### Vectorizaton and slicing


### GPU support


### Performance
* eager exectution vs performance
* `tf.function`
> In general, it's not necessary to decorate each of these smaller functions with tf.function; only use tf.function to decorate high-level computations - for example, one step of training or the forward pass of your model.

# Tensorflow 1 specifics

### Graphs
* So far we did not assign any explicit graph
* The reason is that tensoflow introduces the so called **default graph**
* Any variable specified get's added to the default graph
* However, we may also introduce other graphs, apart from the default graph.

### Node Lifecycle
* All node _values_ are dropped between graph runs (within a session).
* Node values are (intermediate) results
* This implies that node values are not reused (as in spark) per default.
* This leads to  inefficiencies if also intermediate results are of interest. 
* For each (intermediate) result output TF runs an individual graph run.
* In order to avoid redundant evaluations, you need to ask tensorflow to **evaluate all interesting variables within a single graph run**.
* This can be achieved by 



#### Additional Information from the textbook (A. Geron, _Hands on machine learning with scikit-learn and tensoflow_, 2017):

> All node values are dropped between graph runs, except variable values, which are maintained by the
session across graph runs (queues and readers also maintain some state, as we will see in Chapter 12). A
variable starts its life when its initializer is run, and it ends when the session is closed.

> In single-process TensorFlow, multiple sessions do not share any state, even if they reuse the same graph (each session would have its own copy of every variable). In distributed TensorFlow (see Chapter 12), variable state is stored on the servers, not in the sessions, so multiple sessions can share the same variables.


# Todos / Brainstorming
* TF2 has greatly improved name spacing of variables (https://www.tensorflow.org/guide/effective_tf2) 
* How to implement fit predict cleanly? 
* Warnings with Jupyter notebooks
* Visualization of the computational graph

### Possible Examples
* Logistic Regression
* Linear Regression
* Generalized linear models
* Survival Analysis
* Customer Livetime values
* Decision Tree
* Random Forest